<a href="https://colab.research.google.com/github/rs-delve/covid19_datasets/blob/master/age/data/Load_Italy_Cases_from_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%pip install -q tabula-py
import tabula
import pandas as pd

     |████████████████████████████████| 10.4MB 2.6MB/s 


In [40]:
italy_cases_pdfs = {  # Value is tuple(URL, Page Number, Parameter Set Index)
    '30 June 2020': ('https://www.epicentro.iss.it/coronavirus/bollettino/Bollettino-sorveglianza-integrata-COVID-19_30-giugno-2020.pdf', 6, 0),
    '23 June 2020': ('https://www.epicentro.iss.it/coronavirus/bollettino/Bollettino-sorveglianza-integrata-COVID-19_23-giugno-2020.pdf', 7, 0),
    '16 June 2020': ('https://www.epicentro.iss.it/coronavirus/bollettino/Bollettino-sorveglianza-integrata-COVID-19_16-giugno-2020.pdf', 6, 0),
    '9 June 2020': ('https://www.epicentro.iss.it/coronavirus/bollettino/Bollettino-sorveglianza-integrata-COVID-19_9-giugno-2020.pdf', 6, 1),  # Measurements are different!! 
    '3 June 2020': ('https://www.epicentro.iss.it/coronavirus/bollettino/Bollettino-sorveglianza-integrata-COVID-19_3-giugno-2020.pdf', 6, 1),
    '26 May 2020': ('https://www.epicentro.iss.it/coronavirus/bollettino/Bollettino-sorveglianza-integrata-COVID-19_26-maggio-2020.pdf', 6, 1),
    '20 May 2020': ('https://www.epicentro.iss.it/coronavirus/bollettino/Bollettino-sorveglianza-integrata-COVID-19_20-maggio-2020.pdf', 7, 1),  
    '14 May 2020': ('https://www.epicentro.iss.it/coronavirus/bollettino/Bollettino-sorveglianza-integrata-COVID-19_14-maggio-2020.pdf', 7, 1),
    '7 May 2020': ('https://www.epicentro.iss.it/coronavirus/bollettino/Bollettino-sorveglianza-integrata-COVID-19_7-maggio-2020.pdf', 8, 1),
    '28 April 2020': ('https://www.epicentro.iss.it/coronavirus/bollettino/Bollettino-sorveglianza-integrata-COVID-19_28-aprile-2020.pdf', 8, 1),
    '23 April 2020': ('https://www.epicentro.iss.it/coronavirus/bollettino/Bollettino-sorveglianza-integrata-COVID-19_23-aprile-2020.pdf', 7, 1),
    '16 April 2020': ('https://www.epicentro.iss.it/coronavirus/bollettino/Bollettino-sorveglianza-integrata-COVID-19_16-aprile-2020.pdf', 7, 1),
    '9 April 2020': ('https://www.epicentro.iss.it/coronavirus/bollettino/Bollettino-sorveglianza-integrata-COVID-19_9-aprile-2020.pdf', 8, 1),
    '2 April 2020': ('https://www.epicentro.iss.it/coronavirus/bollettino/Bollettino-sorveglianza-integrata-COVID-19_2-aprile-2020.pdf', 6, 2),  # Measurements change again here
    '30 March 2020': ('https://www.epicentro.iss.it/coronavirus/bollettino/Bollettino-sorveglianza-integrata-COVID-19_30-marzo-2020.pdf', 6, 2)
}

measurement_sets = [
    # (top left bottom right) (row_start, row_end), (column boundaries), [Drop columns]
    (2.73*72, 0.35*72, 5.18*72, 5.05*72),  
    (4*72, 0.78*72, 6.44*72, 5.32*72),
    (4.28*72, 0.78*72, 6.59*72, 5.32*72)
]

age_groups = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '>90']

In [43]:
url = 'https://www.epicentro.iss.it/coronavirus/bollettino/Bollettino-sorveglianza-integrata-COVID-19_3-giugno-2020.pdf'
page = 6
param_idx = 1
date = '30 June 2020'

measurements = measurement_sets[param_idx]
print('Date:', date, 'URL:', url, 'Page:', page, 'Param set:', param_idx)

df = tabula.read_pdf(url, output_format='dataframe', pages=page, guess=False, area=measurements, pandas_options={'dtype': 'str', 'header': None}, )
age_df = df[0].iloc[:, [0, 1, -1]]
age_df.columns = ['Age', 'm', 'f']
age_df.Age = age_df.Age.replace('≥90', '>90')
age_df = age_df[age_df.Age.isin(age_groups)]
for age_grp in age_groups:
  assert age_grp in age_df.Age.values, f'Missing age group: {age_grp}'

def convert(s):
  return int(str(s).replace('.', ''))

age_df.m = age_df.m.apply(convert)
age_df.f = age_df.f.apply(convert)

age_df['Date'] = pd.to_datetime(date)
age_df = age_df.set_index(['Date', 'Age']).stack().reset_index().rename(columns={'level_2': 'Sex', 0: 'cases_new'})
age_df

Date: 30 June 2020 URL: https://www.epicentro.iss.it/coronavirus/bollettino/Bollettino-sorveglianza-integrata-COVID-19_3-giugno-2020.pdf Page: 6 Param set: 1


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,Date,Age,Sex,cases_new
0,2020-06-30,0-9,m,1047
1,2020-06-30,0-9,f,938
2,2020-06-30,10-19,m,1799
3,2020-06-30,10-19,f,1771
4,2020-06-30,20-29,m,5826
5,2020-06-30,20-29,f,7416
6,2020-06-30,30-39,m,8187
7,2020-06-30,30-39,f,10094
8,2020-06-30,40-49,m,12722
9,2020-06-30,40-49,f,17720


In [45]:
all_age_dfs = []

for date, (url, page, param_idx) in italy_cases_pdfs.items():
  print('Date:', date, 'URL:', url, 'Page:', page, 'Param set:', param_idx)
  measurements = measurement_sets[param_idx]

  df = tabula.read_pdf(url, output_format='dataframe', pages=page, guess=False, area=measurements, pandas_options={'dtype': 'str', 'header': None})

  age_df = df[0].iloc[:, [0, 1, -1]]
  age_df.columns = ['Age', 'm', 'f']
  age_df.Age = age_df.Age.replace('≥90', '>90')
  age_df = age_df[age_df.Age.isin(age_groups)]
  for age_grp in age_groups:
    assert age_grp in age_df.Age.values, f'Missing age group: {age_grp}'

  def convert(s):
    return int(str(s).replace('.', ''))

  age_df.m = age_df.m.apply(convert)
  age_df.f = age_df.f.apply(convert)

  age_df['Date'] = pd.to_datetime(date)
  age_df = age_df.set_index(['Date', 'Age']).stack().reset_index().rename(columns={'level_2': 'Sex', 0: 'cases_new'})
  all_age_dfs.append(age_df)

Date: 30 June 2020 URL: https://www.epicentro.iss.it/coronavirus/bollettino/Bollettino-sorveglianza-integrata-COVID-19_30-giugno-2020.pdf Page: 6 Param set: 0


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Date: 23 June 2020 URL: https://www.epicentro.iss.it/coronavirus/bollettino/Bollettino-sorveglianza-integrata-COVID-19_23-giugno-2020.pdf Page: 7 Param set: 0


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Date: 16 June 2020 URL: https://www.epicentro.iss.it/coronavirus/bollettino/Bollettino-sorveglianza-integrata-COVID-19_16-giugno-2020.pdf Page: 6 Param set: 0


...

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Date: 2 April 2020 URL: https://www.epicentro.iss.it/coronavirus/bollettino/Bollettino-sorveglianza-integrata-COVID-19_2-aprile-2020.pdf Page: 6 Param set: 2


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Date: 30 March 2020 URL: https://www.epicentro.iss.it/coronavirus/bollettino/Bollettino-sorveglianza-integrata-COVID-19_30-marzo-2020.pdf Page: 6 Param set: 2


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [46]:
pd.concat(all_age_dfs, axis=0)

[         Date    Age Sex  cases_new
 0  2020-06-30    0-9   m       1154
 1  2020-06-30    0-9   f       1035
 2  2020-06-30  10-19   m       1954
 3  2020-06-30  10-19   f       1949
 4  2020-06-30  20-29   m       6204
 5  2020-06-30  20-29   f       7797
 6  2020-06-30  30-39   m       8586
 7  2020-06-30  30-39   f      10508
 8  2020-06-30  40-49   m      13197
 9  2020-06-30  40-49   f      18231
 10 2020-06-30  50-59   m      19867
 11 2020-06-30  50-59   f      23172
 12 2020-06-30  60-69   m      19069
 13 2020-06-30  60-69   f      12987
 14 2020-06-30  70-79   m      19489
 15 2020-06-30  70-79   f      14640
 16 2020-06-30  80-89   m      16777
 17 2020-06-30  80-89   f      24702
 18 2020-06-30    >90   m       3860
 19 2020-06-30    >90   f      15219,          Date    Age Sex  cases_new
 0  2020-06-23    0-9   m       1135
 1  2020-06-23    0-9   f       1021
 2  2020-06-23  10-19   m       1937
 3  2020-06-23  10-19   f       1908
 4  2020-06-23  20-29   m       6117
 

In [49]:
final = pd.concat(all_age_dfs, axis=0)


,Date,Age,Sex,cases_new
0,2020-06-30,0-9,m,1154
1,2020-06-30,0-9,f,1035
2,2020-06-30,10-19,m,1954
3,2020-06-30,10-19,f,1949
4,2020-06-30,20-29,m,6204
...,...,...,...,...
15,2020-03-30,70-79,f,614
16,2020-03-30,80-89,m,7630
17,2020-03-30,80-89,f,650
18,2020-03-30,>90,m,1160


In [50]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [51]:
path = '/gdrive/My Drive/italy_cases_by_age_sex.csv'
with open(path, 'w') as f:
  final.to_csv(f)

print('Wrote', path)

Wrote /gdrive/My Drive/italy_cases_by_age_sex.csv
